### 预处理

In [1]:
# import some packages
import pandas as pd
from collections import Counter
from tflearn.data_utils import pad_sequences
import random
import numpy as np
import h5py
import pickle
print("import package successful...")

D:\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
import package successful...


In [2]:
# read source file as csv
base_path='D:/zhihu_data/data/ieee_zhihu_cup2/'
train_data_x=pd.read_csv(base_path+'question_train_set3.txt',sep='\t', encoding="utf-8")
train_data_y=pd.read_csv(base_path+'question_topic_train_set3.txt',sep='\t', encoding="utf-8")
valid_data_x=pd.read_csv(base_path+'question_eval_set3.txt', sep='\t',encoding="utf-8")

train_data_x=train_data_x.fillna('')
train_data_y=train_data_y.fillna('')
valid_data_x=valid_data_x.fillna('')
print("train_data_x:",train_data_x.shape)
print("train_data_y:",train_data_y.shape)
print("valid_data_x:",valid_data_x.shape)

train_data_x: (2999967, 5)
train_data_y: (2999967, 2)
valid_data_x: (217360, 5)


In [3]:
train_data_x.head()

,question_id,title_char,title_word,desc_char,desc_word
0,6555699376639805223,"c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...","w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...","c335,c101,c611,c189,c97,c144,c147,c101,c15,c76...","w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1..."
1,2887834264226772863,"c44,c110,c101,c286,c106,c150,c101,c892,c632,c1...","w377,w54,w285,w57,w349,w54,w108215,w6,w47986,w...","c1265,c518,c74,c131,c274,c57,c768,c769,c368,c3...","w12508,w1380,w72,w27045,w276,w111"
2,-2687466858632038806,"c15,c768,c769,c1363,c650,c1218,c2361,c11,c90,c...","w875,w15450,w42394,w15863,w6,w95421,w25,w803,w...","c693,c100,c279,c99,c189,c532,c101,c189,c145,c1...","w140340,w54,w48398,w54,w140341,w54,w12856,w54,..."
3,-5698296155734268,"c473,c1528,c528,c428,c295,c15,c101,c188,c146,c...","w8646,w2744,w1462,w9,w54,w138,w54,w50,w110,w14...",,
4,-6719100304248915192,"c190,c147,c105,c219,c220,c101,c647,c219,c220,c...","w380,w54,w674,w133,w54,w134,w614,w54,w929,w307...","c644,c1212,c253,c199,c431,c452,c424,c207,c2,c1...","w4821,w1301,w16003,w928,w1961,w2565,w50803,w11..."


In [10]:
# compute average length of title_char, title_word, desc_char, desc_word

dict_length_columns={'title_char':0,'title_word':0,'desc_char':0,'desc_word':0}
num_examples=len(train_data_x)
train_data_x_small=train_data_x.sample(frac=0.01)
for index, row in train_data_x_small.iterrows():
    title_char_length=len(row['title_char'].split(","))
    title_word_length=len(row['title_word'].split(","))
    desc_char_length=len(row['desc_char'].split(","))
    desc_word_length=len(row['desc_word'].split(","))
    dict_length_columns['title_char']=dict_length_columns['title_char']+title_char_length
    dict_length_columns['title_word']=dict_length_columns['title_word']+title_word_length
    dict_length_columns['desc_char']=dict_length_columns['desc_char']+desc_char_length
    dict_length_columns['desc_word']=dict_length_columns['desc_word']+desc_word_length
dict_length_columns={k:float(v)/float(num_examples*0.01) for k,v in dict_length_columns.items()}
print("dict_length_columns:",dict_length_columns)

# averaged length of a input. total length of all information(words,character of title+desc): 210.
# word of title+desc: 71
# character of title+desc: 139

dict_length_columns: {'title_char': 22.350212519004373, 'title_word': 12.91504206546272, 'desc_char': 116.06157667734344, 'desc_word': 57.72050159218418}


In [11]:
train_data_y.head()

,question_id,topic_ids
0,6555699376639805223,"7739004195693774975,3738968195649774859"
1,2887834264226772863,-3149765934180654494
2,-2687466858632038806,-760432988437306018
3,-5698296155734268,"-6758942141122113907,3195914392210930723"
4,-6719100304248915192,"3804601920633030746,4797226510592237555,435133..."


In [12]:
# average labels for a input
train_data_y_small=train_data_y.sample(frac=0.01)
num_examples=len(train_data_y_small)
num_labels=0
for index, row in train_data_y_small.iterrows():
    topic_ids=row['topic_ids']
    topic_id_list=topic_ids.split(",")
    num_labels+=len(topic_id_list)
average_num_labels=float(num_labels)/float(num_examples)
print("average_num_labels:",average_num_labels)


average_num_labels: 2.3377


In [13]:
word_embedding_object=open(base_path+'unused_current/char_embedding.txt')
lines_wv=word_embedding_object.readlines()
word_embedding_object.close()

In [14]:
len(lines_wv)

11974

In [15]:
lines_wv[0].split(' ')

['11973', '256\n']

In [16]:
#创建word2index表和embedding表
word_embedding_object=open(base_path+'unused_current/char_embedding.txt')
lines_wv=word_embedding_object.readlines()
word_embedding_object.close()
char_list=[]
char_list.extend(['PAD','UNK','SEP'])
PAD_ID=0
UNK_ID=1
EMBED_SIZE = 100
bound = 0.5
embedding = [None] * (int(lines_wv[0].split(' ')[0]) + 3) #长度
embedding[0] = np.zeros(100)
embedding[1] = np.random.rand(100)
embedding[2] = embedding[0] + 0.1
for i, line in enumerate(lines_wv):
    if i==0: continue
    char_embedding_list=line.split(" ")
    char_token=char_embedding_list[0]
    char_list.append(char_token)
    embedding[i+2] = char_embedding_list[1:1+EMBED_SIZE]

embedding = np.array(embedding,dtype=np.float32)
# write to vocab.txt under data/ieee_zhihu_cup2
vocab_path=base_path+'vocab.txt'

vocab_char_object=open(vocab_path,'w')

word2index={}
for i, char in enumerate(char_list):
    if i<10:print(i,char)
    word2index[char]=i
    vocab_char_object.write(char+"\n")
vocab_char_object.close()
print("vocabulary of char generated....")

0 PAD
1 UNK
2 SEP
3 </s>
4 c17
5 c101
6 c11
7 c4
8 c147
9 c85
vocabulary of char generated....


In [17]:
 # generate labels list, and save to file system 
c_labels=Counter()
train_data_y_small=train_data_y[0:100000]#.sample(frac=0.1)
for index, row in train_data_y_small.iterrows():
    topic_ids=row['topic_ids']
    topic_list=topic_ids.split(',')
    c_labels.update(topic_list)

label_list=c_labels.most_common()
label2index={}
label_target_object=open(base_path+'label_set.txt','w')
for i, label_freq in enumerate(label_list):
    label,freq=label_freq
    label2index[label]=i
    label_target_object.write(label+"\n")
    if i<20: print(label,freq)
label_target_object.close()
print("generate label dict successful...")

7476760589625268543 2308
4697014490911193675 1746
-4653836020042332281 1579
-8175048003539471998 1475
-8377411942628634656 1382
-7046289575185911002 1338
-5932391056759866388 1283
2787171473654490487 1145
-7129272008741138808 1085
2587540952280802350 1079
-4931965624608608932 1079
-6748914495015758455 1049
-5513826101327857645 993
2347973810368732059 970
9069451131871918127 958
-8132909213241034354 904
-3517637179126242000 867
-5872443091340192918 834
-3522198575349379632 830
1127459907694805235 829
generate label dict successful...


In [18]:
def transform_multilabel_as_multihot(label_list,label_size):
    """
    convert to multi-hot style
    :param label_list: e.g.[0,1,4], here 4 means in the 4th position it is true value(as indicate by'1')
    :param label_size: e.g.199
    :return:e.g.[1,1,0,1,0,0,........]
    """
    result=np.zeros(label_size)
    #set those location as 1, all else place as 0.
    result[label_list] = 1
    return result

label_list=[0,1,2,10]
label_size=20
label_list_sparse=transform_multilabel_as_multihot(label_list,label_size)
print("label_list_sparse:",label_list_sparse)

label_list_sparse: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
vocabulary_word2index = word2index
def get_X_Y(train_data_x,train_data_y,label_size, test_mode=False):
    """
    get X and Y given input and labels
    input:
    train_data_x:
    train_data_y:
    label_size: number of total unique labels(e.g. 1999 in this task)
    output:
    X,Y
    """
    X=[]
    Y=[]
    if test_mode:
        train_data_x_tiny_test=train_data_x[0:1000] # todo todo todo todo todo todo todo todo todo todo todo todo 
        train_data_y_tiny_test=train_data_y[0:1000] # todo todo todo todo todo todo todo todo todo todo todo todo 
    else:
        train_data_x_tiny_test=train_data_x
        train_data_y_tiny_test=train_data_y

    for index, row in train_data_x_tiny_test.iterrows():
        if index==0: 
            print(row)
            continue
        # get character of title and dssc
        title_char=row['title_char']
        desc_char=row['desc_char']
        # split into list
        title_char_list=title_char.split(',')
        desc_char_list=desc_char.split(",")
        # transform to indices
        title_char_id_list=[vocabulary_word2index.get(x,UNK_ID) for x in title_char_list if x.strip()]
        desc_char_id_list=[vocabulary_word2index.get(x,UNK_ID) for x in desc_char_list if x.strip()]
        # merge title and desc: in the middle is special token 'SEP'
        title_char_id_list.append(vocabulary_word2index['SEP'])
        title_char_id_list.extend(desc_char_id_list)
        X.append(title_char_id_list)
        if index<3: print(index,title_char_id_list)
        if index%100000==0: print(index,title_char_id_list)
    
    #产生单一标签
    for index, row in train_data_y_tiny_test.iterrows():
        if index==0: continue
        topic_id = row['topic_ids'].split(',')[0]
        Y.append(label2index[topic_id])
        if index%100000==0: print(index,";label_index:",label2index[topic_id])
    '''
    for index, row in train_data_y_tiny_test.iterrows():
        if index==0: continue
        topic_ids=row['topic_ids']
        topic_id_list=topic_ids.split(",")
        label_list_dense=[label2index[l] for l in topic_id_list if l.strip()]
        label_list_sparse=transform_multilabel_as_multihot(label_list_dense,label_size)
        Y.append(label_list_sparse)
        if index%100000==0: print(index,";label_list_dense:",label_list_dense)
    '''
    return X,Y

In [20]:
def save_data(cache_file_h5py,cache_file_pickle,word2index,label2index,train_X,train_Y,vaild_X,valid_Y,test_X,test_Y, embedding):
    # train/valid/test data using h5py
    f = h5py.File(cache_file_h5py, 'w')
    f['train_X'] = train_X
    f['train_Y'] = train_Y
    f['vaild_X'] = vaild_X
    f['valid_Y'] = valid_Y
    f['test_X'] = test_X
    f['test_Y'] = test_Y
    f['embedding'] = embedding
    f.close()
    # save word2index, label2index
    with open(cache_file_pickle, 'ab') as target_file:
        pickle.dump((word2index,label2index), target_file)

In [21]:
# generate training/validation/test data using source file and vocabulary/label set.
#  get X,Y---> shuffle and split data----> save to file system.
test_mode=False
label_size=len(label2index)
cache_path_h5py=base_path+'data.h5'
cache_path_pickle=base_path+'vocab_label.pik'
max_sentence_length=200

# step 1: get (X,y) 
X,Y=get_X_Y(train_data_x,train_data_y,label_size,test_mode=test_mode)

# pad and truncate to a max_sequence_length
X = pad_sequences(X, maxlen=max_sentence_length, value=0.)  # padding to max length

# step 2. shuffle, split,
xy=list(zip(X,Y))
random.Random(10000).shuffle(xy)
X,Y=zip(*xy)
X=np.array(X); Y=np.array(Y)
num_examples=len(X)
num_valid=20000
num_valid=20000
num_train=num_examples-(num_valid+num_valid)
train_X, train_Y=X[0:num_train], Y[0:num_train]
vaild_X, valid_Y=X[num_train:num_train+num_valid], Y[num_train:num_train+num_valid]
test_X, test_Y=X[num_train+num_valid:], Y[num_train+num_valid:]
print("num_examples:",num_examples,";X.shape:",X.shape,";Y.shape:",Y.shape)
print("train_X:",train_X.shape,";train_Y:",train_Y.shape,";vaild_X.shape:",vaild_X.shape,";valid_Y:",valid_Y.shape,";test_X:",test_X.shape,";test_Y:",test_Y.shape)

# step 3: save to file system
save_data(cache_path_h5py,cache_path_pickle,word2index,label2index,train_X,train_Y,vaild_X,valid_Y,test_X,test_Y, embedding)
print("save cache files to file system successfully!")

del X,Y,train_X, train_Y,vaild_X, valid_Y,test_X, test_Y, embedding


question_id                                  6555699376639805223
title_char     c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...
title_word     w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...
desc_char      c335,c101,c611,c189,c97,c144,c147,c101,c15,c76...
desc_word      w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...
Name: 0, dtype: object
1 [104, 137, 5, 25, 29, 22, 5, 516, 1386, 191, 661, 6, 188, 909, 1605, 503, 52, 61, 27, 9, 54, 58, 78, 1411, 642, 262, 60, 137, 103, 10, 2, 537, 90, 58, 20, 67, 13, 61, 27, 357, 595, 10]
2 [52, 61, 27, 2146, 556, 1348, 816, 6, 131, 1684, 159, 7, 128, 89, 87, 6, 350, 523, 37, 113, 10, 2, 618, 18, 85, 114, 100, 197, 5, 100, 46, 100, 8, 114, 114, 353, 5, 49, 18, 8, 395, 46, 5, 8, 15, 5, 31, 5, 84, 51, 77, 15, 30, 46, 5, 8, 77, 28, 5, 46, 15, 51, 49, 46, 5, 70, 353, 5, 14, 22, 5, 3656, 5, 14, 21, 5, 84, 51, 77, 15, 30, 46, 5, 339, 736, 78, 663, 233, 30, 15, 15, 49, 179, 32, 32, 33, 33, 33, 29, 73, 8, 46, 15, 40, 51, 395, 24, 28, 46, 29, 51, 40, 73,

1700000 [526, 407, 1122, 560, 7, 12, 177, 88, 96, 6, 229, 427, 10, 2, 26, 647, 12, 465, 526, 407, 104, 1122, 560, 4, 26, 526, 407, 262, 117, 124, 262, 87, 68, 509, 225, 1389, 104, 93, 13, 125, 139, 229, 427, 10, 5, 80, 247, 24, 28, 18, 51, 5, 24, 28, 44, 19, 25, 23, 22, 14, 34, 19, 5, 28, 8, 15, 8, 39, 46, 33, 85, 100, 40, 114, 44, 19, 30, 15, 15, 49, 179, 32, 32, 49, 114, 8, 353, 18, 40, 29, 353, 51, 100, 395, 100, 29, 36, 51, 84, 32, 49, 114, 8, 353, 18, 40, 29, 49, 30, 49, 32, 46, 24, 28, 32, 434, 384, 245, 374, 17, 384, 245, 312, 31, 472, 245, 73, 33, 32, 247, 29, 46, 33, 85, 19, 5, 49, 51, 46, 15, 18, 40, 44, 19, 30, 15, 15, 49, 179, 32, 32, 73, 25, 29, 353, 395, 24, 84, 73, 29, 36, 51, 84, 32, 17, 14, 17, 17, 34, 31, 14, 592, 31, 34, 23, 14, 22, 17, 34, 17, 25, 246, 222, 21, 25, 17, 17, 23, 246, 420, 451, 246, 22, 14, 34, 222, 21, 420, 34, 21, 34, 31, 39, 25, 449, 592, 25, 39, 420, 23, 451, 246, 39, 31, 23, 17, 25, 39, 22, 23, 17, 592, 38, 25, 38, 38, 449, 449, 222, 449, 19, 5, 2